In [7]:
# prepare the data
import pandas as pd

# File paths
features_file = "carc_16comp_x.csv"
labels_file = "carc_16comp_y.csv"

# Load the data
features_df = pd.read_csv(features_file)
labels_df = pd.read_csv(labels_file)

from sklearn.model_selection import train_test_split

labels = labels_df.values.ravel()

# Split the data into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(features_df, labels, test_size=0.2, random_state=42)

# Check shape of data
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((268, 16), (67, 16), (268,), (67,))

In [ ]:
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import StatevectorSampler as Sampler
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit import QuantumCircuit

NUM_QUBITS = 16

adhoc_feature_map = ZZFeatureMap(feature_dimension=NUM_QUBITS, reps=2, entanglement="linear").decompose()
# feature_map = QuantumCircuit(16)
#Using feature_map here causes the parameter dimensions mismatch issue previously seen.
#decomposing the ZZFeatureMap directly is not a problem however
# feature_map.compose(adhoc_feature_map.decompose())

sampler = Sampler()

fidelity = ComputeUncompute(sampler=sampler)

adhoc_kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=adhoc_feature_map)

adhoc_feature_map.draw("mpl")

adhoc_feature_map.decompose().depth()


31

In [9]:
from qiskit_machine_learning.algorithms import QSVC

qsvc = QSVC(quantum_kernel=adhoc_kernel)

qsvc.fit(X_train, y_train)

qsvc_score = qsvc.score(X_test, y_test)

print(f"QSVC classification test score: {qsvc_score}")

ValueError: x_vec and class feature map have incompatible dimensions.
x_vec has 16 dimensions, but feature map has 8.